In [1]:
!python -V

Python 3.9.12


In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [4]:
!pip install pyarrow

     |████████████████████████████████| 29.4 MB 2.8 MB/s eta 0:00:01


In [2]:
trip_data = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')

In [3]:
trip_data.head(5)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [4]:
len(trip_data)

1154112

In [5]:
trip_data['trip_duration'] = (trip_data['dropOff_datetime'] -  trip_data['pickup_datetime']).astype('timedelta64[s]')/60

In [6]:
trip_data.head(5)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,trip_duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,110.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667


In [8]:
trip_data['trip_duration'].mean()

19.1672240937939

In [10]:
trip_data = trip_data[(trip_data.trip_duration>=1) & (trip_data.trip_duration<=60)]

In [11]:
trip_data['PUlocationID'] = trip_data['PUlocationID'].fillna(-1)
trip_data['DOlocationID'] = trip_data['DOlocationID'].fillna(-1)

/tmp/ipykernel_2673/1324908322.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_data['PUlocationID'] = trip_data['PUlocationID'].fillna(-1)
/tmp/ipykernel_2673/1324908322.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_data['DOlocationID'] = trip_data['DOlocationID'].fillna(-1)


In [12]:
trip_data.head(5)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,trip_duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,9.050000


In [13]:
(trip_data['PUlocationID'] == -1).sum()/len(trip_data)*100

83.52732770722618

In [14]:
categorical = ['PUlocationID', 'DOlocationID']
target = ['trip_duration']

In [15]:
dv = DictVectorizer()

In [16]:
trip_data[categorical] = trip_data[categorical].astype(str)

/tmp/ipykernel_2673/989405720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_data[categorical] = trip_data[categorical].astype(str)


In [17]:
train_dicts = trip_data[categorical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)
y_train = trip_data[target].values

In [18]:
X_train.shape

(1109826, 525)

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.5285191072072

In [20]:
trip_data_test = pd.read_parquet('./data/fhv_tripdata_2021-02.parquet')

In [21]:
trip_data_test.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037


In [22]:
trip_data_test['trip_duration'] = (trip_data_test['dropOff_datetime'] -  trip_data_test['pickup_datetime']).astype('timedelta64[s]')/60

In [23]:
trip_data_test.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,trip_duration
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014,92.000000
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021,10.666667
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021,14.566667
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021,7.950000
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037,13.800000


In [25]:
trip_data_test = trip_data_test[(trip_data_test.trip_duration>=1) & (trip_data_test.trip_duration<=60)]
trip_data_test = trip_data_test[trip_data_test.DOlocationID!=225.0]
trip_data_test['PUlocationID'] = trip_data_test['PUlocationID'].fillna(-1)
trip_data_test['DOlocationID'] = trip_data_test['DOlocationID'].fillna(-1)

In [26]:
trip_data_test.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,trip_duration
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021,10.666667
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021,14.566667
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021,7.950000
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1.0,61.0,None,B00037,8.966667
6,B00112,2021-02-01 00:30:25,2021-02-01 00:57:23,-1.0,26.0,None,B00112,26.966667


In [27]:
trip_data_test[categorical] = trip_data_test[categorical].astype(str)
val_dicts = trip_data_test[categorical].to_dict(orient='records')

#dv = DictVectorizer()
X_val = dv.fit_transform(val_dicts)
y_val = trip_data_test[target].values

In [28]:
len(dv.feature_names_)

525

In [29]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.7815149939228